In [2]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install kiwipiepy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment
from kiwipiepy import Kiwi
from collections import Counter
import re

src = r"naverkeyword.xlsx"
out = r"naverkeyword_result_v1.xlsx"

# ========키워드 합계 & 사이즈 ===============
  
# header_row는 1로 나타낼 수 있음
def find_col_by_header(ws, keywords, header_row = 1) :
    # keywords = ["연관키워드", "월간검색수 (PC)", "월간검색수 (MOBILE)", ... , "월평균노출 광고수"]
    for col in range(1, ws.max_column + 1) : 
        v = ws.cell(row=header_row, column=col).value
        if not v :
            continue
        s = str(v).replace(" ", "") # 월간검색수 (PC) -> 월간검색수(PC)

        ok = True
        for kw in keywords :
            if kw.replace(" ", "") not in s :
                ok = False
                break
        if ok : 
            return col
    return None
    
wb = load_workbook(src)
ws = wb.active 

pc_col = (
    find_col_by_header(ws, ["월간검색수", "PC"]) 
    or find_col_by_header(ws, ["월간검색수", "피씨"]) 
    or 2
)
mo_col = (find_col_by_header(ws, ["월간검색수", "모바일"]) 
          or find_col_by_header(ws, ["월간검색수", "Mobile"]) 
          or 3
)

total_col = 4
size_col = 5

ws.cell(row=1, column=total_col).value = "전책검색량(PC+모바일)" 
ws.cell(row=1, column=size_col).value = "키워드규모(전체검색량)"

for c in [total_col, size_col] : 
    cell = ws.cell(row=1, column=c)
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center")

def to_int(x) : 
    if x is None :
        return 0
    if isinstance(x, (int, float)) :
        return int(x)
    s = str(x).strip()
    if s in ("<10", "< 10") :
        return 0
    s = s.replace(",", "")
    try :
        return int(float(s))
    except :
        return 0

for r in range(2, ws.max_row + 1) :
    pc = to_int(ws.cell(row=r, column=pc_col).value)
    mo = to_int(ws.cell(row=r, column=mo_col).value)
    total = pc + mo
    ws.cell(row=r, column=total_col).value = total

    if total >= 3000 :
        size = "대형(3000+)"
    elif total >= 1000 :
        size = "중형(1000+)"
    else :
        size = "소형(<1000)"
    ws.cell(row=r, column=size_col).value = size

# 3.14 -> 실수 (float), 9 10 -> 정수 (int)   
# "1,439.34" => int(에러) // int(float()) -> 1439(가능)

ws.auto_filter.ref = f"A1:{ws.cell(row=1, column=max(ws.max_column, size_col)).coordinate}"

# ========키워드 형태소 분석 ============

fixed_catetory = {
    "원터치" : ("타입", "원터치"),
    "드롭" : ("타입", "드롭"),
    "클립" : ("타입", "클립"),
    "명품" : ("브랜드", "명품"),
    "브랜드" : ("브랜드", "브랜드"),
    "명품" : ("성별", "여자")
}

brands = {
    "샤넬", "스톤헨지", "스와로브스키", "티파니", "까르띠에", "불가리", "반클리프", "구찌"
}



def classify_token(token: str) :
    t = str(token).strip()  

    if not t : 
        return ("기타", "")

    if t in fixed_catetory : 
        return fixed_catetory[t]

    if t in brands :
        return ("브랜드", t)

    return ("기타", "")

if "키워드_빈도" in wb.sheetnames :
    del wb ["키워드_빈도"]
ws_kw = wb.create_sheet("키워드_빈도")

keyword_col = 1
texts = [
    row[0] for row in ws.iter_rows(min_row=2, max_col=1, values_only=True) 
    if row and row[0]
]
all_text = " ".join(map(str, texts))

kiwi = Kiwi()
tokens = kiwi.tokenize(all_text)

noun_tags = {"NNG", "NNP", "NNB", "NR"}
noun_tokens = [t.form for t in tokens if t.tag in noun_tags]

regex_tokens = re.findall(r"[A-Za-z]*\d+[A-Za-z]+|\d+[A-Za-z]*", all_text) 

stopwords = {"엑스", "구매", "다음", "때문", "만족", "생각"}

valid_one = {"금", "은", "동", "링", "침"}

combined = noun_tokens + regex_tokens
filtered = [t for t in combined if (len(str(t)) > 1 or t in valid_one) and t not in stopwords]

count = Counter(filtered)

ws_kw.append(["키워드","빈도","카테고리","세부범주"])
for cell in ws_kw[1] : 
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center")

for kw, freq in count.most_common() :
    cat, sub = classify_token(kw)
    ws_kw.append([kw, freq, cat, sub])

ws_kw.auto_filter.ref = f"A1:D{ws_kw.max_row}"

# 14K, gold14K, 18K, 925, k14
# 탐색보고서, 검색창 -> 검색하려고 할 때
# 특정 경로값을 포함한 주소 찾아야 할 때
# https://www.naver.com/category/shopping?utm_campaign=""&

# ["귀걸이", "진주귀걸이", "진주목걸이"....], tuple ("귀걸이",) -> 인덱스 값, iter의 속성, 반복문, 추가&삭제 불가
# 귀걸이 진주귀걸이 진주목걸이 20대의악세서리.....
# 명사 : 귀걸이, 진주, 목걸이 -> 
# [{"tag": "N","form": "귀걸이"} }, {...} ] 
# NNG = Noun Noun General : 일반명사 -> 귀걸이, 반지, 목걸이, 디자인
# NNP = Noun Noun Proper : 고유명사 -> 샤넬, 스와로브스키, 티파니
# NNB = Noun Noun Bound : 의존명사 -> 개, 명, 번, 가지, 수
# NR = Noun Numeral : 수사 -> 하나, 둘, 셋, 넷, 세, 열

# 데이터 관련 분야 -> 깃허브, 정규표현식 (GA4 & BigQuery)

wb.save(out)
print("완료 : ", out)

완료 :  naverkeyword_result_v1.xlsx
